In [6]:
import os
import hydra
import numpy as np
import random
import time

from torch.nn import functional as F
from cliport import tasks
from cliport.dataset import RavensDataset
from cliport.environments.environment import Environment
import IPython
import random
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from cliport.dataset import RavensDataset
import copy

In [7]:
cfg = {}
cfg['n'] = 1000
cfg['task'] = 'build-car'
cfg['mode'] = 'test'
cfg['save_data'] = True
cfg['assets_root'] = '/home/cosmos/VScode Projects/coglab/GenSim/cliport/environments/assets'
cfg['data_dir'] = '/home/cosmos/VScode Projects/coglab/GenSim/data'
cfg['disp'] = False
cfg['shared_memory'] = False
cfg['record'] = {}
cfg['record']['save_video'] = True
cfg['record']['save_video_path'] = '/home/cosmos/VScode Projects/coglab/GenSim/videos'
cfg['record']['add_text'] = False
cfg['record']['add_task_text'] = True
cfg['record']['fps'] = 20
cfg['record']['video_height'] = 640
cfg['record']['video_width'] = 720

cfg['dataset'] = {}
cfg['dataset']['type'] = 'single' # 'single' or 'multi'
cfg['dataset']['images'] = True
cfg['dataset']['cache'] = True # load episodes to memory instead of reading from disk
cfg['dataset']['augment'] = {}
cfg['dataset']['augment']['theta_sigma'] = 60

In [8]:
env = Environment(
        cfg['assets_root'],
        disp=cfg['disp'],
        shared_memory=cfg['shared_memory'],
        hz=480,
        record_cfg=cfg['record']
    )
cfg['task'] = cfg['task'].replace("_", "-")
task = tasks.names[cfg['task']]()
task.mode = cfg['mode']
record = cfg['record']['save_video']
save_data = cfg['save_data']

text argument:/home/cosmos/VScode Projects/coglab/GenSim/cliport/environments/assets
int args: [

In [4]:
agent = task.oracle(env)

# Run ORACLE baseline

In [5]:
episode_id = 0

np.random.seed(episode_id)
random.seed(episode_id)

env.set_task(task)
obs = env.reset()
info = env.info
reward = 0
episode, total_reward = [], 0
data_path = os.path.join(cfg['data_dir'], "{}-{}".format(cfg['task'], task.mode))
dataset = RavensDataset(data_path, cfg, n_demos=0, augment=False)

if record:
    env.start_rec(f'{dataset.n_episodes+1:06d}')

# Rollout expert policy
for _ in range(task.max_steps):
    act = agent.act(obs, info)
    episode.append((obs, act, reward, info))
    lang_goal = info['lang_goal']
    obs, reward, done, info = env.step(act)
    total_reward += reward
    print(f'Total Reward: {total_reward:.3f} | Done: {done} | Goal: {lang_goal}')
    if done:
        break
if record:
    env.end_rec()

[libx264 @ 0x6017980] -qscale is ignored, -crf is recommended.


object 1 match with target 1 rew: 0.167
Total Reward: 0.167 | Done: False | Goal: Firstly, create the base of the car by positioning two red blocks side by side.
object 0 match with target 0 rew: 0.167
object 1 match with target 1 rew: 0.333
Total Reward: 0.333 | Done: False | Goal: Firstly, create the base of the car by positioning two red blocks side by side.
Total Reward: 0.333 | Done: False | Goal: Then, add the car body by stacking a blue block on top of the base.
Total Reward: 0.333 | Done: False | Goal: Then, add the car body by stacking a blue block on top of the base.
Total Reward: 0.333 | Done: False | Goal: Then, add the car body by stacking a blue block on top of the base.
object 0 match with target 0 rew: 0.333
Total Reward: 0.667 | Done: False | Goal: Then, add the car body by stacking a blue block on top of the base.
object 2 match with target 2 rew: 0.083
Total Reward: 0.750 | Done: False | Goal: For the wheels, place a black cylinder on each side of the base blocks.
ob

## Run CLIPORT checkpoint

In [ ]:
%env GENSIM_ROOT=$(pwd)

env: GENSIM_ROOT=$(pwd)


In [ ]:
import os

os.environ['GENSIM_ROOT'] = os.getcwd()
os.environ['CLIPORT_ROOT'] = os.getcwd()

In [ ]:
# set GPU
# %env CUDA_VISIBLE_DEVICES=0

import os
import sys
import json

import numpy as np
from cliport import tasks
from cliport import agents
from cliport.utils import utils

import torch
import cv2
from cliport.dataset import RavensDataset
from cliport.environments.environment import Environment
from torch.utils.data import DataLoader

%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

/home/cosmos/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pybullet build time: Jan 29 2024 08:20:00


In [ ]:
train_demos = 1000 # number training demonstrations used to train agent
n_eval = 1 # number of evaluation instances
mode = 'test' # val or test

agent_name = 'cliport'
model_task = 'multi-language-conditioned' # multi-task agent conditioned with language goals

model_folder = 'cliport_quickstart' # path to pre-trained checkpoint
ckpt_name = 'steps=400000-val_loss=0.00014655.ckpt' # name of checkpoint to load

draw_grasp_lines = True
affordance_heatmap_scale = 30

### Uncomment the task you want to evaluate on ###
# eval_task = 'align-rope'
# eval_task = 'assembling-kits-seq-seen-colors'
# eval_task = 'assembling-kits-seq-unseen-colors'
# eval_task = 'packing-shapes'
# eval_task = 'packing-boxes-pairs-seen-colors'
# eval_task = 'packing-boxes-pairs-unseen-colors'
# eval_task = 'packing-seen-google-objects-seq'
# eval_task = 'packing-unseen-google-objects-seq'
# eval_task = 'packing-seen-google-objects-group'
# eval_task = 'packing-unseen-google-objects-group'
# eval_task = 'put-block-in-bowl-seen-colors'
# eval_task = 'put-block-in-bowl-unseen-colors'
eval_task = 'stack-block-pyramid-seq-seen-colors'
# eval_task = 'stack-block-pyramid-seq-unseen-colors'
# eval_task = 'separating-piles-seen-colors'
# eval_task = 'separating-piles-unseen-colors'
# eval_task = 'towers-of-hanoi-seq-seen-colors'
# eval_task = 'towers-of-hanoi-seq-unseen-colors'

In [ ]:
root_dir = os.environ['GENSIM_ROOT']
assets_root = os.path.join(root_dir, 'cliport/environments/assets/')
config_file = 'my_eval.yaml' 

vcfg = utils.load_hydra_config(os.path.join(root_dir, f'cliport/cfg/{config_file}'))
vcfg['data_dir'] = os.path.join(root_dir, 'data')
vcfg['mode'] = mode

vcfg['model_task'] = model_task
vcfg['eval_task'] = eval_task
vcfg['agent'] = agent_name

# Model and training config paths
model_path = os.path.join(root_dir, model_folder)
# vcfg['train_config'] = f"{model_path}/cliport/{vcfg['model_task']}-{vcfg['agent']}-n{train_demos}-train/.hydra/config.yaml"
# vcfg['model_path'] = f"{model_path}/{vcfg['model_task']}-{vcfg['agent']}-n{train_demos}-train/checkpoints/"
vcfg['root_dir'] = root_dir
tcfg = utils.load_hydra_config(vcfg['train_config'])

# Load dataset
ds = RavensDataset(os.path.join(vcfg['data_dir'], f'{vcfg["eval_task"]}-{vcfg["mode"]}'), 
                   tcfg, 
                   n_demos=n_eval,
                   augment=False)
data_loader = DataLoader(ds, shuffle=False,
                    pin_memory=False,
                    num_workers=1)

eval_run = 0
name = '{}-{}-{}-{}'.format(vcfg['eval_task'], vcfg['agent'], n_eval, eval_run)
print(f'\nEval ID: {name}\n')

# Initialize agent
utils.set_seed(eval_run, torch=True)
agent = agents.names[vcfg['agent']](name, tcfg, data_loader, data_loader)

# Load checkpoint
ckpt_path = os.path.join(vcfg['model_path'], ckpt_name)
print(f'\nLoading checkpoint: {ckpt_path}')
agent.load(ckpt_path)

Requested training on 1 demos, but only 0 demos exist in the dataset path: /home/cosmos/VScode Projects/coglab/GenSim/data/stack-block-pyramid-seq-seen-colors-test.

Eval ID: stack-block-pyramid-seq-seen-colors-cliport-1-0

Attn FCN - Stream One: plain_resnet_lat, Stream Two: clip_lingunet_lat, Stream Fusion: add
Transport FCN - Stream One: plain_resnet_lat, Stream Two: clip_lingunet_lat, Stream Fusion: conv
Agent: stack-block-pyramid-seq-seen-colors-cliport-1-0, Logging: False

Loading checkpoint: /home/cosmos/VScode Projects/coglab/GenSim/cliport/cliport_quickstart/multi-language-conditioned-cliport-n1000-train/checkpoints/steps=400000-val_loss=0.00014655.ckpt


RuntimeError: Error(s) in loading state_dict for TwoStreamClipLingUNetLatTransporterAgent:
	Unexpected key(s) in state_dict: "attention.attn_stream_one.layer5.0.conv1.weight", "attention.attn_stream_one.layer5.0.conv2.weight", "attention.attn_stream_one.layer5.0.conv3.weight", "attention.attn_stream_one.layer5.0.shortcut.0.weight", "attention.attn_stream_one.layer5.1.conv1.weight", "attention.attn_stream_one.layer5.1.conv2.weight", "attention.attn_stream_one.layer5.1.conv3.weight", "attention.attn_stream_one.layer6.0.conv1.weight", "attention.attn_stream_one.layer6.0.conv2.weight", "attention.attn_stream_one.layer6.0.conv3.weight", "attention.attn_stream_one.layer6.0.shortcut.0.weight", "attention.attn_stream_one.layer6.1.conv1.weight", "attention.attn_stream_one.layer6.1.conv2.weight", "attention.attn_stream_one.layer6.1.conv3.weight", "attention.attn_stream_two.up1.conv.double_conv.0.weight", "attention.attn_stream_two.up1.conv.double_conv.0.bias", "attention.attn_stream_two.up1.conv

In [ ]:
# Initialize environment and task.
env = Environment(
    assets_root,
    disp=False,
    shared_memory=False,
    hz=480,
    record_cfg=False
)


In [ ]:
episode = 0
num_eval_instances = min(n_eval, ds.n_episodes)

for i in range(num_eval_instances):
    print(f'\nEvaluation Instance: {i + 1}/{num_eval_instances}')
    
    # Load episode
    episode, seed = ds.load(i)
    goal = episode[-1]
    total_reward = 0
    np.random.seed(seed)

    # Set task
    task_name = vcfg['eval_task']
    task = tasks.names[task_name]()
    task.mode = mode
    
    # Set environment
    env.seed(seed)
    env.set_task(task)
    obs = env.reset()
    info = env.info
    reward = 0
    
    step = 0
    done = False
    
    # Rollout
    while (step <= task.max_steps) and not done:
        print(f"Step: {step} ({task.max_steps} max)")
        
        # Get batch
        if step == task.max_steps-1:
            batch = ds.process_goal((obs, None, reward, info), perturb_params=None)
        else:
            batch = ds.process_sample((obs, None, reward, info), augment=False)

        fig, axs = plt.subplots(2, 2, figsize=(13, 7))
        
        # Get color and depth inputs
        img = batch['img']
        img = torch.from_numpy(img)
        color = np.uint8(img.detach().cpu().numpy())[:,:,:3]
        color = color.transpose(1,0,2)
        depth = np.array(img.detach().cpu().numpy())[:,:,3]
        depth = depth.transpose(1,0)
        
        # Display input color
        axs[0,0].imshow(color)
        axs[0,0].axes.xaxis.set_visible(False)
        axs[0,0].axes.yaxis.set_visible(False)
        axs[0,0].set_title('Input RGB')
        
        # Display input depth
        axs[0,1].imshow(depth)
        axs[0,1].axes.xaxis.set_visible(False)
        axs[0,1].axes.yaxis.set_visible(False)        
        axs[0,1].set_title('Input Depth')
        
        # Display predicted pick affordance
        axs[1,0].imshow(color)
        axs[1,0].axes.xaxis.set_visible(False)
        axs[1,0].axes.yaxis.set_visible(False)
        axs[1,0].set_title('Pick Affordance')
        
        # Display predicted place affordance
        axs[1,1].imshow(color)
        axs[1,1].axes.xaxis.set_visible(False)
        axs[1,1].axes.yaxis.set_visible(False)
        axs[1,1].set_title('Place Affordance')
        
        # Get action predictions
        l = str(info['lang_goal'])
        act = agent.act(obs, info, goal=None)
        pick, place = act['pick'], act['place']
        
        # Visualize pick affordance
        pick_inp = {'inp_img': batch['img'], 'lang_goal': l}
        pick_conf = agent.attn_forward(pick_inp)
        logits = pick_conf.detach().cpu().numpy()

        pick_conf = pick_conf.detach().cpu().numpy()
        argmax = np.argmax(pick_conf)
        argmax = np.unravel_index(argmax, shape=pick_conf.shape)
        p0 = argmax[:2]
        p0_theta = (argmax[2] * (2 * np.pi / pick_conf.shape[2])) * -1.0
    
        line_len = 30
        pick0 = (pick[0] + line_len/2.0 * np.sin(p0_theta), pick[1] + line_len/2.0 * np.cos(p0_theta))
        pick1 = (pick[0] - line_len/2.0 * np.sin(p0_theta), pick[1] - line_len/2.0 * np.cos(p0_theta))

        if draw_grasp_lines:
            axs[1,0].plot((pick1[0], pick0[0]), (pick1[1], pick0[1]), color='r', linewidth=1)
        
        # Visualize place affordance
        place_inp = {'inp_img': batch['img'], 'p0': pick, 'lang_goal': l}
        place_conf = agent.trans_forward(place_inp)

        place_conf = place_conf.permute(1, 2, 0)
        place_conf = place_conf.detach().cpu().numpy()
        argmax = np.argmax(place_conf)
        argmax = np.unravel_index(argmax, shape=place_conf.shape)
        p1_pix = argmax[:2]
        p1_theta = (argmax[2] * (2 * np.pi / place_conf.shape[2]) + p0_theta) * -1.0
        
        line_len = 30
        place0 = (place[0] + line_len/2.0 * np.sin(p1_theta), place[1] + line_len/2.0 * np.cos(p1_theta))
        place1 = (place[0] - line_len/2.0 * np.sin(p1_theta), place[1] - line_len/2.0 * np.cos(p1_theta))

        if draw_grasp_lines:
            axs[1,1].plot((place1[0], place0[0]), (place1[1], place0[1]), color='g', linewidth=1)
        
        # Overlay affordances on RGB input
        pick_logits_disp = np.uint8(logits * 255 * affordance_heatmap_scale).transpose(1,0,2)
        place_logits_disp = np.uint8(np.sum(place_conf, axis=2)[:,:,None] * 255 * affordance_heatmap_scale).transpose(1,0,2)    

        pick_logits_disp_masked = np.ma.masked_where(pick_logits_disp < 0, pick_logits_disp)
        place_logits_disp_masked = np.ma.masked_where(place_logits_disp < 0, place_logits_disp)

        axs[1][0].imshow(pick_logits_disp_masked, alpha=0.75)
        axs[1][1].imshow(place_logits_disp_masked, cmap='viridis', alpha=0.75)
        
        print(f"Lang Goal: {str(info['lang_goal'])}")
        plt.show()
        
        # Act with the predicted actions
        obs, reward, done, info = env.step(act)
        step += 1
        
    if done:
        print("Done. Success.")
    else:
        print("Max steps reached. Task failed.")

In [ ]:
"""Ravens main training script."""

import os
import pickle
import json

import numpy as np
import hydra
from hydra import compose, initialize
from cliport import agents
from cliport import dataset
from cliport import tasks
from cliport.utils import utils
from cliport.environments.environment import Environment


def main():
    # Load train cfg
    # tcfg = utils.load_hydra_config(vcfg['train_config'])
    with initialize(config_path='./cliport/cfg', version_base="1.2"):
        vcfg = compose(config_name='my_eval')
    tcfg = utils.load_hydra_config(vcfg['train_config'])



    # Initialize environment and task.
    env = Environment(
        vcfg['assets_root'],
        disp=vcfg['disp'],
        shared_memory=vcfg['shared_memory'],
        hz=480,
        record_cfg=vcfg['record']
    )

    # Choose eval mode and task.
    mode = vcfg['mode']
    eval_task = vcfg['eval_task']
    if mode not in {'train', 'val', 'test'}:
        raise Exception("Invalid mode. Valid options: train, val, test")

    # Load eval dataset.
    dataset_type = vcfg['type']
    if 'multi' in dataset_type:
        ds = dataset.RavensMultiTaskDataset(vcfg['data_dir'],
                                            tcfg,
                                            group=eval_task,
                                            mode=mode,
                                            n_demos=vcfg['n_demos'],
                                            augment=False)
    else:
        ds = dataset.RavensDataset(os.path.join(vcfg['data_dir'], f"{eval_task}-{mode}"),
                                   tcfg,
                                   n_demos=vcfg['n_demos'],
                                   augment=False)

    all_results = {}
    name = '{}-{}-n{}'.format(eval_task, vcfg['agent'], vcfg['n_demos'])

    # Save path for results.
    json_name = f"multi-results-{mode}.json" if 'multi' in vcfg['model_path'] else f"results-{mode}.json"
    save_path = vcfg['save_path']
    print(f"Save path for results: {save_path}")
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    save_json = os.path.join(save_path, f'{name}-{json_name}')

    # Load existing results.
    existing_results = {}
    if os.path.exists(save_json):
        with open(save_json, 'r') as f:
            existing_results = json.load(f)

    # Make a list of checkpoints to eval.
    ckpts_to_eval = list_ckpts_to_eval(vcfg, existing_results)

    # Evaluation loop
    print(f"Evaluating: {str(ckpts_to_eval)}")
    for ckpt in ckpts_to_eval:
        model_file = os.path.join(vcfg['model_path'], ckpt)

        if not os.path.exists(model_file) or not os.path.isfile(model_file):
            print(f"Checkpoint not found: {model_file}")
            continue
        elif not vcfg['update_results'] and ckpt in existing_results:
            print(f"Skipping because of existing results for {model_file}.")
            continue

        results = []
        mean_reward = 0.0

        # Run testing for each training run.
        for train_run in range(vcfg['n_repeats']):

            # Initialize agent.
            utils.set_seed(train_run, torch=True)
            agent = agents.names[vcfg['agent']](name, tcfg, None, ds)

            # Load checkpoint
            agent.load(model_file)
            print(f"Loaded: {model_file}")

            record = vcfg['record']['save_video']
            n_demos = vcfg['n_demos']

            # Run testing and save total rewards with last transition info.
            for i in range(0, n_demos):
                print(f'Test: {i + 1}/{n_demos}')
                episode, seed = ds.load(i)
                goal = episode[-1]
                total_reward = 0
                np.random.seed(seed)

                # set task
                if 'multi' in dataset_type:
                    task_name = ds.get_curr_task()
                    task = tasks.names[task_name]()
                    print(f'Evaluating on {task_name}')
                else:
                    task_name = vcfg['eval_task']
                    task = tasks.names[task_name]()

                task.mode = mode
                env.seed(seed)
                env.set_task(task)
                obs = env.reset()
                info = env.info
                reward = 0

                # Start recording video (NOTE: super slow)
                if record:
                    video_name = f'{task_name}-{i+1:06d}'
                    if 'multi' in vcfg['model_task']:
                        video_name = f"{vcfg['model_task']}-{video_name}"
                    env.start_rec(video_name)

                for _ in range(task.max_steps):
                    act = agent.act(obs, info, goal)
                    lang_goal = info['lang_goal']
                    print(f'Lang Goal: {lang_goal}')
                    obs, reward, done, info = env.step(act)
                    total_reward += reward
                    print(f'Total Reward: {total_reward:.3f} | Done: {done}\n')
                    if done:
                        break

                results.append((total_reward, info))
                mean_reward = np.mean([r for r, i in results])
                print(f'Mean: {mean_reward} | Task: {task_name} | Ckpt: {ckpt}')

                # End recording video
                if record:
                    env.end_rec()

            all_results[ckpt] = {
                'episodes': results,
                'mean_reward': mean_reward,
            }

        # Save results in a json file.
        if vcfg['save_results']:

            # Load existing results
            if os.path.exists(save_json):
                with open(save_json, 'r') as f:
                    existing_results = json.load(f)
                existing_results.update(all_results)
                all_results = existing_results

            with open(save_json, 'w') as f:
                json.dump(all_results, f, indent=4)


def list_ckpts_to_eval(vcfg, existing_results):
    ckpts_to_eval = []

    # Just the last.ckpt
    if vcfg['checkpoint_type'] == 'last':
        last_ckpt = 'last.ckpt'
        ckpts_to_eval.append(last_ckpt)

    # Validation checkpoints that haven't been already evaluated.
    elif vcfg['checkpoint_type'] == 'val_missing':
        checkpoints = sorted([c for c in os.listdir(vcfg['model_path']) if "steps=" in c])
        ckpts_to_eval = [c for c in checkpoints if c not in existing_results]

    # Find the best checkpoint from validation and run eval on the test set.
    elif vcfg['checkpoint_type'] == 'test_best':
        result_jsons = [c for c in os.listdir(vcfg['results_path']) if "results-val" in c]
        if 'multi' in vcfg['model_task']:
            result_jsons = [r for r in result_jsons if "multi" in r]
        else:
            result_jsons = [r for r in result_jsons if "multi" not in r]

        if len(result_jsons) > 0:
            result_json = result_jsons[0]
            with open(os.path.join(vcfg['results_path'], result_json), 'r') as f:
                eval_res = json.load(f)
            best_checkpoint = 'last.ckpt'
            best_success = -1.0
            for ckpt, res in eval_res.items():
                if res['mean_reward'] > best_success:
                    best_checkpoint = ckpt
                    best_success = res['mean_reward']
            print(best_checkpoint)
            ckpt = best_checkpoint
            ckpts_to_eval.append(ckpt)
        else:
            print("No best val ckpt found. Using last.ckpt")
            ckpt = 'last.ckpt'
            ckpts_to_eval.append(ckpt)

    # Load a specific checkpoint with a substring e.g: 'steps=10000'
    else:
        print(f"Looking for: {vcfg['checkpoint_type']}")
        checkpoints = [c for c in os.listdir(vcfg['model_path']) if vcfg['checkpoint_type'] in c]
        checkpoint = checkpoints[0] if len(checkpoints) > 0 else ""
        ckpt = checkpoint
        ckpts_to_eval.append(ckpt)

    return ckpts_to_eval


if __name__ == '__main__':
    main()

/home/cosmos/.local/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'my_eval': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


Requested training on 100 demos, but only 0 demos exist in the dataset path: /home/cosmos/VScode Projects/coglab/GenSim/data/multi-language-conditioned-test.text argument:/home/cosmos/VScode Projects/coglab/GenSim/cliport/environments/assets/
int args: [
Save path for results: /home/cosmos/VScode Projects/coglab/GenSim/cliport/cliport_quickstart/multi-language-conditioned-cliport-n1000-train/checkpoints/
Evaluating: ['steps=400000-val_loss=0.00014655.ckpt']


AttributeError: 'NoneType' object has no attribute 'dataset'